In [2]:
import numpy as np

import tensorflow_hub as hub
import tensorflow as tf

import re

from naps.pipelines.read_naps import read_naps_dataset

C:\ProgramData\Anaconda3\envs\synth\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\ProgramData\Anaconda3\envs\synth\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\ProgramData\Anaconda3\envs\synth\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\ProgramData\Anaconda3\envs\synth\lib\site-packages\tensorflow\python\f

In [3]:
global elmo

def init():
    global elmo
    elmo = hub.Module("https://tfhub.dev/google/elmo/3", trainable=True)


def get_words(text):
    sentences_dirty = text.split('.')

    max_len = 0

    sentences = []
    for sent in sentences_dirty:
        sent = re.sub('[,;-]', '', sent)
        if len(sent) > 0:
            sentences.append(sent)
            splitted = sent.split()
            if len(splitted) > max_len:
                max_len = len(splitted)

    words = []
    mask = []
    masked_words = []
    for sent in sentences:
        splitted = sent.split()
        for i in range(max_len):
            try:
                words.append(splitted[i])
            except:
                words.append('_')

    for word in words:
        if word == "_":
            mask.append(False)
        else:
            mask.append(True)
            masked_words.append(word)
    return sentences, mask

def get_embeddings(text):
    global elmo
    sentences, mask = get_words(text)

    embeddings = elmo(
        sentences,
        signature="default",
        as_dict=True)["elmo"]


    with tf.compat.v1.Session() as sess:
        sess.run(tf.compat.v1.global_variables_initializer())
        sess.run(tf.compat.v1.tables_initializer())
        x = sess.run(embeddings)
    #print(x)
    embs = x.reshape(-1, 1024)
    masked_embs = embs[mask]

    return np.array(masked_embs)

In [4]:
def make_the_tree_good(bad_tree):
    # Zwracana zmienna
    good_tree = []

    # Warunek zakończenia rekurencji
    if type(bad_tree) is not list:
        good_tree = bad_tree

    # Warunek zaistnienia rekurencji
    else:
        for x in bad_tree:
            mttg = make_the_tree_good(x)

            # Dobranie sposobu dołączenia do wynikowej listy
            if type(mttg) is list:
                good_tree.extend(mttg)
            else:
                good_tree.append(str(mttg))

    return good_tree

In [7]:
def get_unique_tokens():
    ds, _, test = read_naps_dataset()
    tokens_total = []
    with ds:
        
        for d in ds:
            if "is_partial" in d and d["is_partial"]:
                continue
            
            tokens = list(set(list(make_the_tree_good(d["code_tree"]["funcs"]))))
            tokens_total += tokens
            
    with test:
        
        for d in test:
            if "is_partial" in d and d["is_partial"]:
                continue
            
            tokens = list(set(list(make_the_tree_good(d["code_tree"]["funcs"]))))
            tokens_total += tokens

    tokens_total = list(set(tokens_total))
    print(tokens_total) 
    print(len(tokens_total)) #1850

    with open('tokens.txt', 'w') as f:
        for token in tokens_total:
            f.write("%s\n" % token)

def get_tokens():
    with open('tokens.txt', 'r') as f:
        tokens = f.read().splitlines()
        return tokens

def generate_output(tokens, unique):
    values = np.zeros((len(tokens), len(unique)))
    for i in range(len(tokens)):
        index = unique.index(str(tokens[i]))
        values[i][index] = 1
    return values

def generate_tokens(output, unique):
    output = np.array(output)
    tokens = []
    for out in output:
        idx = np.argmax(out)
        tokens.append(unique[idx])
    return tokens

def even_embeddings(embed, n):
    if len(embed) < n:
        diff =  n - len(embed)
        embed = np.vstack([embed, np.zeros((diff,len(embed[0])))])
    return embed

def even_tokens(tokens, n):
    tokens = np.concatenate((['START'],tokens,['END'] ))
    if len(tokens) < n:
        diff =  n - len(tokens)
        tokens = np.concatenate((tokens, np.full(diff, '')))
    return tokens

In [8]:
init()
unique = get_tokens()

ds, _, test = read_naps_dataset()
X_train = []
Y_train = []
X_test = []
Y_test = []

Y_tokens = []
Y_tokens2 = []
max_embs_len = 0
max_tokens_len = 0

stopper = 1
stopper2 = 1
counter = 0

with ds:
    for d in ds:
        if "is_partial" in d and d["is_partial"]:
            continue
        counter += 1
        masked_embs = get_embeddings(' '.join(d["text"]))
        if len(masked_embs) > max_embs_len:
            max_embs_len = len(masked_embs)
        X_train.append(masked_embs)    
        tokens = make_the_tree_good(d["code_tree"]["funcs"])
        #print(make_the_tree_good(d["code_tree"]["funcs"]))
        if len(tokens) > max_tokens_len:
            max_tokens_len = len(tokens)
        Y_tokens.append(tokens)
        #break
        if counter == stopper:
            break
counter = 0
with test:
    for t in test:
        if "is_partial" in t and t["is_partial"]:
            continue
        counter += 1
        masked_embs = get_embeddings(' '.join(t["text"]))
        if len(masked_embs) > max_embs_len:
            max_embs_len = len(masked_embs)
        X_test.append(masked_embs)    
        tokens = make_the_tree_good(t["code_tree"]["funcs"])
        #print(make_the_tree_good(d["code_tree"]["funcs"]))
        if len(tokens) > max_tokens_len:
            max_tokens_len = len(tokens)
        Y_tokens2.append(tokens)
        if counter == stopper2:
            break
        #break


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [9]:
X_train_2 = []
#print(X_train)
for ex in X_train:
    X_train_2.append(even_embeddings(ex, max_embs_len))

for ex in Y_tokens:
    #print(even_tokens(ex,max_tokens_len))
    Y_train.append(generate_output(even_tokens(ex,max_tokens_len), unique))

X_test_2 = []
#print(X_train)
for ex in X_test:
    X_test_2.append(even_embeddings(ex, max_embs_len))

for ex in Y_tokens2:
    #print(even_tokens(ex,max_tokens_len))
    Y_test.append(generate_output(even_tokens(ex,max_tokens_len), unique))

In [10]:
X_train_2 = np.array(X_train_2)
X_test_2 = np.array(X_test_2)
Y_train = np.array(Y_train)
Y_test = np.array(Y_test)

In [ ]:
with open('data/X_train_2.npy', 'wb') as f:
    np.save(f, X_train_2)
    
with open('data/X_test_2.npy', 'wb') as f:
    np.save(f, X_test_2)
    
with open('Y_train.npy', 'wb') as f:
    np.save(f, Y_train)
    
with open('Y_test.npy', 'wb') as f:
    np.save(f, Y_test)